In [1]:
import datetime
import json
import pandas as pd
import glob
import os
import numpy as np

today_str=str(datetime.datetime.now().date())

writer_floder="/home/jian/Projects/Big_Lots/Weather/Forecast_Validation/"+today_str+"/"
try:
    os.stat(writer_floder)
except:
    os.mkdir(writer_floder)

In [2]:
forecast_file_list=glob.glob("/home/jian/Projects/Big_Lots/Weather/Json_data/forecast/"+"*.json")

actual_file_list=glob.glob("/home/jian/Projects/Big_Lots/Weather/Json_data/daily/"+"*.json")

In [3]:
forecast_file_Q1_df=pd.DataFrame()
Saturday_actual_file_Q1_df=pd.DataFrame()
Sunday_actual_file_Q1_df=pd.DataFrame()
i=1
for file in forecast_file_list:
    location=file
    date=datetime.datetime.strptime(file.split("/")[len(file.split("/"))-1][0:10],"%Y-%m-%d").date()
    
    if (date>=datetime.date(2018,2,4)) & (date<=datetime.date(2018,5,5)) & (date.weekday()==2): #Wednesday only
        forecast_file_Q1_df=forecast_file_Q1_df.append(pd.DataFrame({"Date_Monday":date,'Date_Saturday':date+datetime.timedelta(days=3),"forecast_file":location},index=[i]))
        i=i+1
i=1
for file in actual_file_list:
    location=file
    date=datetime.datetime.strptime(file.split("/")[len(file.split("/"))-1][0:10],"%Y-%m-%d").date()
    saturday_date=date+datetime.timedelta(days=-1)
    if (date>=datetime.date(2018,2,4)) & (date<=datetime.date(2018,5,5)) & (date.weekday()==5): #Saturday only
        Saturday_actual_file_Q1_df=Saturday_actual_file_Q1_df.append(pd.DataFrame({"Date_Saturday":date,"actual_file_Saturday":location},index=[i]))
        i=i+1
        
    if (date>=datetime.date(2018,2,4)) & (date<=datetime.date(2018,5,6)) & (date.weekday()==6): #Sunday only
        Sunday_actual_file_Q1_df=Sunday_actual_file_Q1_df.append(pd.DataFrame({"Date_Saturday":saturday_date,"actual_file_Sunday":location},index=[i]))
        i=i+1
all_3_files_Q1=pd.merge(forecast_file_Q1_df,Saturday_actual_file_Q1_df,on="Date_Saturday",how="inner")
all_3_files_Q1=pd.merge(all_3_files_Q1,Sunday_actual_file_Q1_df,on="Date_Saturday",how="inner")


In [4]:
inclusion_stores=pd.read_excel("/home/jian/Projects/Big_Lots/Q1_Post/BL_Sales YoY_JL_20180618.xlsx",sheetname="Inclusion Stores",skiprows=1,dtype=str)
store_zips=pd.read_excel("/home/jian/Projects/Big_Lots/Other_Input/all_store_DMA.xlsx",dtype=str)[['location_id','zip']]
store_zips['zip']=store_zips['zip'].apply(lambda x: x.zfill(5))
inclusion_stores=pd.merge(inclusion_stores,store_zips,on="location_id",how="left")
inclusion_stores=inclusion_stores[['location_id','zip']].rename(columns={"zip":"zip_cd"})
inclusion_stores=inclusion_stores[inclusion_stores['location_id']!='1769']

In [5]:
all_forecast_group_value=[]
all_forecast_desc_value=[]

all_actual_group_value=[]
all_actual_desc_value=[]

for i in range(len(all_3_files_Q1)):
    json_actual_Saturday=json.load(open(all_3_files_Q1['actual_file_Saturday'][i],"r"))
    json_actual_Sunday=json.load(open(all_3_files_Q1['actual_file_Sunday'][i],"r"))
    json_forecast=json.load(open(all_3_files_Q1['forecast_file'][i],"r"))
    
    for zip_cd in inclusion_stores['zip_cd'].unique().tolist():
        if (zip_cd in list(json_forecast.keys())) & (zip_cd in list(json_actual_Saturday.keys())) & (zip_cd in list(json_actual_Sunday.keys())):
            weather_forecast_list=json_forecast[zip_cd]['list'][23]['weather']
            weather_actual_list_Saturday=json_actual_Saturday[zip_cd]['weather']
            weather_actual_list_Sunday=json_actual_Sunday[zip_cd]['weather']

            for j in range(len(weather_forecast_list)):
                weather_forecast_group=weather_forecast_list[j]['main']
                # weather_forecast_desc=weather_forecast_list[j]['description']
                all_forecast_group_value=list(set(all_forecast_group_value+[weather_forecast_group]))
                # all_forecast_desc_value=list(set(all_forecast_desc_value+[weather_forecast_desc]))
            
            for j in range(len(weather_actual_list_Saturday)):    
                weather_actual_group_Saturday=weather_actual_list_Saturday[j]['main']
                # weather_actual_desc=weather_actual_list[j]['description']
                all_actual_group_value=list(set(all_actual_group_value+[weather_actual_group_Saturday]))
                # all_actual_desc_value=list(set(all_actual_desc_value+[weather_actual_desc]))
                
            for j in range(len(weather_actual_list_Sunday)):    
                weather_actual_group_Sunday=weather_actual_list_Sunday[j]['main']
                # weather_actual_desc=weather_actual_list[j]['description']
                all_actual_group_value=list(set(all_actual_group_value+[weather_actual_group_Sunday]))
                # all_actual_desc_value=list(set(all_actual_desc_value+[weather_actual_desc]))
        else:
            print(zip_cd,all_3_files_Q1['Date_Saturday'][i])
            
        
        
        

80110 2018-04-21
35806 2018-04-28


In [6]:
all_forecast_group_value

['Rain', 'Snow', 'Clouds', 'Clear']

In [7]:
group_weight_rank=pd.read_excel("/home/jian/Projects/Big_Lots/Weather/Q1_Weather_Counts/Q1_inclusion_store_all_weather_type_ranked.xlsx",sheetname="all_weather_group_list")

group_weight_rank['Severity']=group_weight_rank['Severity'].astype(int)
group_weight_rank['Rank']=group_weight_rank['Rank'].astype(int)
group_weight=group_weight_rank[['all_type_group','Severity']]
group_weight_dict=group_weight[['all_type_group', 'Severity']].set_index('all_type_group').T.to_dict()

group_rank=group_weight_rank[['all_type_group','Rank']]
group_rank_dict=group_rank[['all_type_group', 'Rank']].set_index('all_type_group').T.to_dict()

In [8]:
output_validation=pd.DataFrame()

index_num=1
for i in range(len(all_3_files_Q1)):
    json_actual_Saturday=json.load(open(all_3_files_Q1['actual_file_Saturday'][i],"r"))
    json_actual_Sunday=json.load(open(all_3_files_Q1['actual_file_Sunday'][i],"r"))
    json_forecast=json.load(open(all_3_files_Q1['forecast_file'][i],"r"))
    
    date=all_3_files_Q1['Date_Saturday'][i]
    for zip_cd in inclusion_stores['zip_cd'].unique().tolist():

        if (zip_cd in list(json_forecast.keys())) & (zip_cd in list(json_actual_Saturday.keys()))& (zip_cd in list(json_actual_Sunday.keys())):
            forecast_time=datetime.datetime.fromtimestamp(json_forecast[zip_cd]['list'][23]['dt']) 
            actual_time_Saturday=datetime.datetime.fromtimestamp(json_actual_Saturday[zip_cd]['dt']) 
            actual_time_Sunday=datetime.datetime.fromtimestamp(json_actual_Sunday[zip_cd]['dt']) 
            weather_forecast_list=json_forecast[zip_cd]['list'][23]['weather']
            weather_actual_list_Saturday=json_actual_Saturday[zip_cd]['weather']
            weather_actual_list_Sunday=json_actual_Sunday[zip_cd]['weather']
            
            all_forecast_group_value_zip=[]
            #all_forecast_desc_value_zip=[]
            all_actual_group_value_zip_Saturday=[]
            all_actual_group_value_zip_Sunday=[]
            #all_actual_desc_value_zip_Sautrday=[]
            #all_actual_desc_value_zip_Sunday=[]
            
            for j in range(len(weather_forecast_list)):
                weather_forecast_group=weather_forecast_list[j]['main']
                #weather_forecast_desc=weather_forecast_list[j]['description']
                all_forecast_group_value_zip=list(set(all_forecast_group_value_zip+[weather_forecast_group]))
                #all_forecast_desc_value_zip=list(set(all_forecast_desc_value_zip+[weather_forecast_desc]))
            
            for j in range(len(weather_actual_list_Saturday)):    
                weather_actual_group=weather_actual_list_Saturday[j]['main']
                #weather_actual_desc=weather_actual_list[j]['description']
                all_actual_group_value_zip_Saturday=list(set(all_actual_group_value_zip_Saturday+[weather_actual_group]))
                #all_actual_desc_value_zip=list(set(all_actual_desc_value_zip+[weather_actual_desc]))
            for j in range(len(weather_actual_list_Sunday)):    
                weather_actual_group=weather_actual_list_Sunday[j]['main']
                #weather_actual_desc=weather_actual_list[j]['description']
                all_actual_group_value_zip_Sunday=list(set(all_actual_group_value_zip_Sunday+[weather_actual_group]))
                #all_actual_desc_value_zip=list(set(all_actual_desc_value_zip+[weather_actual_desc]))
            
            all_forecast_group_severity_zip=[]
            all_actual_group_severity_zip_Saturday=[]
            all_actual_group_severity_zip_Sunday=[]
            
            all_forecast_group_rank_zip=[]
            all_actual_group_rank_zip_Saturday=[]
            all_actual_group_rank_zip_Sunday=[]
            
            selected_havest_forecast_group_value_zip=np.nan
            selected_havest_actual_group_value_zip_Saturday=np.nan
            selected_havest_actual_group_value_zip_Sunday=np.nan
            
            
            selected_havest_forecast_rank_value_zip=np.nan
            selected_havest_actual_rank_value_zip_Saturday=np.nan
            selected_havest_actual_rank_value_zip_Sunday=np.nan
            
            
            for k in range(len(all_forecast_group_value_zip)):
                forecast_severity_zip=group_weight_dict[all_forecast_group_value_zip[k]]['Severity']
                all_forecast_group_severity_zip=list(set(all_forecast_group_severity_zip+[forecast_severity_zip]))
                
                forecast_rank_zip=group_rank_dict[all_forecast_group_value_zip[k]]['Rank']
                all_forecast_group_rank_zip=list(set(all_forecast_group_rank_zip+[forecast_rank_zip]))
                
                if k==0:
                    selected_havest_forecast_group_value_zip=all_forecast_group_value_zip[0]
                    
                else:
                    if group_rank_dict[all_forecast_group_value_zip[k]]['Rank']>group_rank_dict[selected_havest_forecast_group_value_zip]['Rank']:
                        selected_havest_forecast_group_value_zip=all_forecast_group_value_zip[k]
                    
                
                
            for k in range(len(all_actual_group_value_zip_Saturday)):
                actual_severity_zip=group_weight_dict[all_actual_group_value_zip_Saturday[k]]['Severity']
                all_actual_group_severity_zip_Saturday=list(set(all_actual_group_severity_zip_Saturday+[actual_severity_zip]))
                
                actual_rank_zip=group_rank_dict[all_actual_group_value_zip_Saturday[k]]['Rank']
                all_actual_group_rank_zip_Saturday=list(set(all_actual_group_rank_zip_Saturday+[actual_rank_zip]))
                if k==0:
                    selected_havest_actual_group_value_zip_Saturday=all_actual_group_value_zip_Saturday[0]
                else:
                    if group_rank_dict[all_actual_group_value_zip_Saturday[k]]['Rank']>group_rank_dict[selected_havest_actual_group_value_zip_Saturday]['Rank']:
                        selected_havest_actual_group_value_zip_Saturday=all_actual_group_value_zip_Saturday[k]
                        
                        
            for k in range(len(all_actual_group_value_zip_Sunday)):
                actual_severity_zip=group_weight_dict[all_actual_group_value_zip_Sunday[k]]['Severity']
                all_actual_group_severity_zip_Sunday=list(set(all_actual_group_severity_zip_Sunday+[actual_severity_zip]))
                
                actual_rank_zip=group_rank_dict[all_actual_group_value_zip_Sunday[k]]['Rank']
                all_actual_group_rank_zip_Sunday=list(set(all_actual_group_rank_zip_Sunday+[actual_rank_zip]))
                if k==0:
                    selected_havest_actual_group_value_zip_Sunday=all_actual_group_value_zip_Sunday[0]
                else:
                    if group_rank_dict[all_actual_group_value_zip_Sunday[k]]['Rank']>group_rank_dict[selected_havest_actual_group_value_zip_Sunday]['Rank']:
                        selected_havest_actual_group_value_zip_Sunday=all_actual_group_value_zip_Sunday[k]
                        
            
            forecast_max_severity_zip=max(all_forecast_group_severity_zip)
            actual_max_severity_zip_Saturday=max(all_actual_group_severity_zip_Saturday)
            actual_max_severity_zip_Sunday=max(all_actual_group_severity_zip_Sunday)
            
            forecast_max_rank_zip=max(all_forecast_group_rank_zip)
            actual_max_rank_zip_Saturday=max(all_actual_group_rank_zip_Saturday)
            actual_max_rank_zip_Sunday=max(all_actual_group_rank_zip_Sunday)
            
            df_app=pd.DataFrame({"zip_cd":zip_cd,"Saturday":date,"Forecast_Time":forecast_time,"Actual_Time_Saturday":actual_time_Saturday,"Actual_Time_Sunday":actual_time_Sunday,
                                 "Forecast_Severity":forecast_max_severity_zip,"Actual_Severity_Saturday":actual_max_severity_zip_Saturday,"Actual_Severity_Sunday":actual_max_severity_zip_Sunday,
                                "Forecast_Weather_Type":selected_havest_forecast_group_value_zip,"Actual_Weather_Type_Saturday":selected_havest_actual_group_value_zip_Saturday,"Actual_Weather_Type_Sunday":selected_havest_actual_group_value_zip_Sunday,
                                "Forecast_Rank":forecast_max_rank_zip,"Actual_Rank_Saturday":actual_max_rank_zip_Saturday,"Actual_Rank_Sunday":actual_max_rank_zip_Sunday},index=[index_num])
            index_num=index_num+1
            output_validation=output_validation.append(df_app)
        else:
            print(zip_cd,all_3_files_Q1['Date_Saturday'][i])
            
output_validation=output_validation[['zip_cd','Saturday','Forecast_Time','Actual_Time_Saturday','Actual_Time_Sunday','Forecast_Weather_Type','Actual_Weather_Type_Saturday','Actual_Weather_Type_Sunday',
                                     'Forecast_Severity','Actual_Severity_Saturday','Actual_Severity_Sunday','Forecast_Rank','Actual_Rank_Saturday','Actual_Rank_Sunday']]
'''
output_validation['Selected_Weekend_Weather']=np.where() #To be added on Monday
output_validation['Selected_Weekend_Rank']=np.where()
output_validation['Selected_Weekend_Severity']=np.where()
'''



80110 2018-04-21
35806 2018-04-28


"\noutput_validation['Selected_Weekend_Weather']=np.where() #To be added on Monday\noutput_validation['Selected_Weekend_Rank']=np.where()\noutput_validation['Selected_Weekend_Severity']=np.where()\n"

In [9]:
temp_Weekend_type=output_validation[['zip_cd','Saturday','Actual_Weather_Type_Saturday','Actual_Weather_Type_Sunday']]
temp_Weekend_type=temp_Weekend_type.drop_duplicates(['zip_cd','Saturday'])
temp_Weekend_type=temp_Weekend_type.reset_index()
del temp_Weekend_type['index']
temp_Weekend_type['Selected_Weekend_Weather']=np.nan
for i in range(len(temp_Weekend_type)):
    if group_rank_dict[temp_Weekend_type['Actual_Weather_Type_Sunday'][i]]['Rank'] > group_rank_dict[temp_Weekend_type['Actual_Weather_Type_Saturday'][i]]['Rank']:
        temp_Weekend_type['Selected_Weekend_Weather'][i]=temp_Weekend_type['Actual_Weather_Type_Sunday'][i]
    else:
        temp_Weekend_type['Selected_Weekend_Weather'][i]=temp_Weekend_type['Actual_Weather_Type_Saturday'][i]
del temp_Weekend_type['Actual_Weather_Type_Saturday']
del temp_Weekend_type['Actual_Weather_Type_Sunday']

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [10]:
output_validation=pd.merge(output_validation,temp_Weekend_type,on=['zip_cd','Saturday'],how="left")
output_validation['Selected_Weekend_Rank']=output_validation[['Actual_Rank_Saturday','Actual_Rank_Sunday']].max(axis=1)
output_validation['Selected_Weekend_Severity']=output_validation[['Actual_Severity_Saturday','Actual_Severity_Sunday']].max(axis=1)

In [11]:
output_validation['Diff_Severity']=output_validation['Selected_Weekend_Severity']-output_validation['Forecast_Severity']
output_validation=pd.merge(output_validation,inclusion_stores,on="zip_cd",how="outer")

In [12]:
sales=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2018-07-07.csv",dtype=str)
sales['week_end_date']=sales['week_end_date'].apply(lambda x:datetime.datetime.strptime(x,"%Y-%m-%d").date())
sales['sales']=sales['sales'].astype(float)
sales_2018=sales[(sales['week_end_date']>=datetime.date(2018,1,1)) & (sales['week_end_date']<=datetime.date(2018,5,31))]
sales_2018=sales_2018[['location_id','week_end_date','sales']]
sales_2018=sales_2018.rename(columns={"week_end_date":"Saturday","sales":"Sales_2018"})
sales_2017=sales[(sales['week_end_date']>=datetime.date(2017,1,1)) & (sales['week_end_date']<=datetime.date(2017,5,31))]
sales_2017=sales_2017[['location_id','week_end_date','sales']]
sales_2017=sales_2017.rename(columns={"sales":"Sales_2017"})
sales_2017['Saturday']=sales_2017['week_end_date'].apply(lambda x: x+datetime.timedelta(days=52*7))
sales_2017=sales_2017[['location_id','Saturday','Sales_2017']]

sales_YoY=pd.merge(sales_2018,sales_2017,on=['location_id','Saturday'],how="inner")

In [13]:
output_validation=pd.merge(output_validation,sales_YoY,on=['location_id','Saturday'],how="left")

In [14]:
unique_weeks=str(len(output_validation['Saturday'].unique()))

writer=pd.ExcelWriter(writer_floder+"BL_2018 Q1 Forecast 3 days validation Wed Sat_Sun_JL_"+today_str+".xlsx",engine="xlsxwriter")
output_validation.to_excel(writer,"all_"+unique_weeks+"_weeks",index=False)
for Saturday,group in output_validation.groupby(['Saturday']):
    group.to_excel(writer,str(Saturday),index=False)
writer.save()